<a href="https://colab.research.google.com/github/samuramirez/cellmigration/blob/master/segmentation_1_class_cell_indiv_tiff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Segmentation one class (cell)

### Mount Google Drive (Colab can see Drive files) and authenticate so that we can interact with GCP via SDK

In [1]:
try:
  from google.cloud import storage
  client = storage.Client();
except Exception as e:
  print(e)
  print("error: no credentials, ignoring");

### Installing and loading packages 

In [2]:
#!nvidia-smi

In [3]:
import pandas as pd
#pip install --upgrade pandas

In [4]:
pd.__version__

'1.3.5'

In [5]:
pip install torch==1.7.1 torchvision==0.8.2

  Using cached torch-1.7.1-cp37-cp37m-manylinux1_x86_64.whl (776.8 MB)
  Using cached torchvision-0.8.2-cp37-cp37m-manylinux1_x86_64.whl (12.8 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0
    Uninstalling torch-1.10.0:
      Successfully uninstalled torch-1.10.0
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: 'sbd.cpython-37.pyc'
Consider using the `--user` option or check the permissions.

Note: you may need to restart the kernel to use updated packages.


In [6]:
import torch
import torchvision
print(torch.__version__)


1.7.1


In [7]:
print(torch.version.cuda)

10.2


In [8]:
torch.cuda.is_available()

/opt/conda/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


False

In [9]:
#torch.cuda.get_device_name(0)

In [10]:
#!pip install -q torch==1.7.0 torchvision

In [11]:
#!pip install fastai==2.0.13 --no-deps


In [12]:
#!pip install fastcore==1.0.13
!pip install fastcore

In [13]:
import fastcore


In [14]:
#!pip install -Uqq fastbook

In [15]:
!pip install fastai --upgrade

  Using cached fastai-2.7.12-py3-none-any.whl (233 kB)
  Using cached fastcore-1.5.29-py3-none-any.whl (67 kB)
  Using cached pydantic-1.8.2-cp37-cp37m-manylinux2014_x86_64.whl (10.1 MB)
  Using cached typing_extensions-3.10.0.2-py3-none-any.whl (26 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.1.1
    Uninstalling typing_extensions-4.1.1:
      Successfully uninstalled typing_extensions-4.1.1
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.9.0
    Uninstalling pydantic-1.9.0:
      Successfully uninstalled pydantic-1.9.0
  Attempting uninstall: fastcore
    Found existing installation: fastcore 1.3.29
    Uninstalling fastcore-1.3.29:
      Successfully uninstalled fastcore-1.3.29
  Attempting uninstall: fastai
    Found existing installation: fastai 2.5.3
    Uninstalling fastai-2.5.3:
      Successfully uninstalled fastai-2.5.3
ERROR: pip's dependency resolver does not currently take into account all t

In [16]:
import fastai
fastai.__version__

'2.7.12'

In [17]:
!pip install scikit-image 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 56.4 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 KB 42.2 MB/s eta 0:00:00
  Using cached tifffile-2021.11.2-py3-none-any.whl (178 kB)


In [18]:
print(fastcore.__version__)


1.3.29


In [19]:
from fastai.vision.all import *

In [20]:
from skimage.io import imread, imsave, imshow
from skimage.exposure import rescale_intensity
import os
import numpy as np
from pathlib import Path
import re
import ntpath
from datetime import datetime
from fastprogress.core import format_time
from IPython.utils.io import capture_output
!ls

310_requirements.txt
Untitled.ipynb
foothold.txt
gcp_transfer
logs
matt_segmentation_1_class_cell_indiv_tiff_context_splitting_gsutil_external_processing.ipynb
segmentation_1_class_cell_indiv_tiff_context_splitting_gsutil_external_processing.ipynb
src
tutorials


# Segmentation Parameters


## Basic Parameters
other shared settings between training and separation

In [21]:
#@markdown What object type you are segmenting
segmentation_target = "Separate - Cell + Nucleus models (two output folders)" #@param ["Nucleus", "Cell", "Separate - Cell + Nucleus models (two output folders)", "Combined - Cell + Nucleus models", "Separate - Single Model (two output folders)", "Combined - Single Model","Custom Models"]


#@markdown local folder where all images and masks to be transferred to and from the GCP will be stored
GCP_transfer_folder = "gcp_transfer" #@param {type:"string"}
GCP_transfer_folder = Path(GCP_transfer_folder);
if not os.path.exists(GCP_transfer_folder):
  os.makedirs(GCP_transfer_folder);

gsutil_dest_folder = GCP_transfer_folder;

#@markdown regex used to decipher filenames (Leave the same for metamorph outputs)
filename_regex = "p[0-9]*_s([0-9]+)_t([0-9]+).*\\.(tif|tiff|TIF|TIFF)" #@param {type:"string"}

#@markdown whether the notebook is being run in a location with persistent file storage (local or GCE VM), as opposed to something like google colab. Will be set to False automatically if google colab is detected.
persistent_files = True; #@param {type:"boolean"}
try:
    from colab import drive
    persistent_files = False;
except Exception:
    pass;

#@markdown not implemented for training yet; will put checkpoint outputs into a log folder
process_logging = True; #@param {type:"boolean"}
if process_logging:
    import logging
    log_folder = Path("logs");
    if not os.path.exists(log_folder): os.mkdir(log_folder);
    foldername = "log_" + str(datetime.now()) + ".txt";
    log_filepath = log_folder/foldername;
    logging.basicConfig(filename=log_filepath, level=logging.DEBUG);
    # del log_folder;

## Cloud Storage Parameters
Filenames and parameters for using google cloud storage

In [22]:
#@markdown The name of the bucket to use (exclude the gs://)
bucket = Path("optotaxisbucket") #@param {type:"string"}

# #@markdown the name of the high level directory in the bucket to which movie segmentation files wil be uploaded
# GCP_parent_dir = "movie_segmentation" #@param {type:"string"}
# GCP_parent_dir = Path(bucket)/GCP_parent_dir
# #NOTE: whenever gsutil is used with GCP_parent_dir, you must add gs://{directory}

#@markdown the path to the folder in the bucket (exclude the bucket name) where models are stored and will be exported to after training
modelsfolder = 'models/fastai' #@param {type:"string"}
gcp_modelsfolder = bucket/modelsfolder;
local_modelsfolder = GCP_transfer_folder/os.path.basename(modelsfolder);
del modelsfolder; #for testing purposes to make sure no old code is using this

## Image Preparation
shared settings between training and segmentation image/mask processing

In [23]:
#@markdown Number of slices (columns/rows) to divide input images into; for the math, pleast check https://www.desmos.com/calculator/t3cyflvlef
x_slices = 5 #@param {type:"integer"}
y_slices =  5 #@param {type:"integer"}
#@markdown dx, dy are the extra context around the segmented center in both directions
dx =  42#@param {type:"integer"}
dy =  32#@param {type:"integer"}
#@markdown x and y crop are how much to straight remove from the image to make the sizes able to be subdivided nicely
x_crop = 0 #@param {type:"integer"}
y_crop = 0 #@param {type:"integer"}

context_bounds = [dy,dx]*2 #assuming x and y symmetrical, not always true -- fix?
crop = [y_crop,x_crop]*2 #both of these are negative y, negative x, positive y, positive x

#@markdown whether to rescale images to their minimum and maximum pixel values; not used if external processing
auto_rescale = True; #@param {type:"bool"}

## Training-Specific Settings
parameters and paths only for training

In [24]:
#@markdown path to the input images within the GCP bucket (exclude the bucket name)
training_images="training_images" #@param {type:"string"}
gcp_training_images=bucket/training_images;
local_training_images = GCP_transfer_folder/training_images;
del training_images; #for testing purposes to make sure no old code is using this

#@markdown path to the input masks within the GCP bucket (exclude the bucket name)
training_masks="/content/gdrive/Othercomputers/My PC/segmentation_iteration_testing/processing/training_masks" #@param {type:"string"}
gcp_training_masks=bucket/training_masks
local_training_masks = GCP_transfer_folder/training_masks;
del training_masks; #for testing purposes to make sure no old code is using this

#@markdown base model that will be used to train from
inmodelname = 'seg_nuc_062719_s_1_2_110619_bleb.pkl' #@param {type:"string"}
#@markdown where the model will be exported to after training
outmodelname = 'iter3_2_nuc_scratch_test.pkl' #@param {type:"string"}

## Segmentation-Specific Settings:
parameters and paths only for segmenting an experiment


In [25]:
experiment = "2023.10.10 OptoTiam Exp 70"

#@markdown path to the input images in the GCP bucket (exclude the bucket name)
segmentation_images=f"movies/{experiment}/{experiment}"  #@param {type:"string"}
gcp_segmentation_images=bucket/segmentation_images;
local_segmentation_images = GCP_transfer_folder/Path(segmentation_images).name;
del segmentation_images; #for testing purposes to make sure no old code is using this

#@markdown folder name where output masks will be deposited in the GCP bucket gcp directory. If segmentation target is "Cell", "Nucleus", or either of the "Separate" options, this will output to the subfolders "Nucleus" and "Cell" respectively. If target is "Combined", will output to the "Combined" subfolder
segmentation_output_masks=f'movie_segmentation/{experiment}/segmentation_output_masks/' #@param {type:"string"}
gcp_segmentation_output_masks=bucket/segmentation_output_masks;
local_segmentation_output_masks=GCP_transfer_folder/Path(segmentation_output_masks).name;
del segmentation_output_masks; #for testing purposes to make sure no old code is using this

print("out file:",gcp_segmentation_output_masks)

#@markdown whether to send output folders to (individual) zip files; if persistent_files is false, this will incur a cost of downloading any nonpersistent files from the cloud and zipping locally
zip_output = True; #@param {type:"boolean"}

#@markdown whether splitting and stitching is already done or should be done by the program
external_splitting_stitching = False; #param {type:"boolean"}

#@markdown name/path in the gcp models folder to the nucleus model for segmentation. This field will not be used if segmentation target is Cell or * - Single Model
nucleus_model = "iter3_4_nuc_continue.pkl" #param {type:"string"}
nucleus_model = local_modelsfolder/nucleus_model;

#@mardown name/path in the gcp models folder to the cell model for segmentation. This field will not be used if segmentation target is Nucleus or * - Single Model
cell_model = "iter1_10_continue.pkl" #param {type:"string"}
cell_model = local_modelsfolder/cell_model;

#@mardown name/path in the gcp models folder to the combined nucleus + cell model for segmentation. This field will not be used if segmentation target is Cell, Nucleus, or Separate - Cell + Nucleus models
combined_model = "" #param {type:"string"}
combined_model = local_modelsfolder/combined_model;

#@markdown dict of model_name:output_folder_name items to be used if segmentation target is Custom Models. Input files will be processed once per model, and the outputs will be saved to the appropriate output file.
#@markdown Advanced usage: if either model_name or output_folder_name is a list/tuple, the segmentation outputs will be combined into one file or split into separate files, respectively
custom_models = {"iter1_1_seg_redo.pkl":"Iter1","iter1_2_seg_redo.pkl":"Iter2","iter1_3_seg_redo.pkl":"Iter3","iter1_4_seg_redo.pkl":"Iter4","iter1_6_context_4_continue.pkl":"Iter6","iter1_8_continue.pkl":"Iter8"} #param {type:"raw"}
relative_custom_models = {};
for key,val in custom_models.items():
    if isinstance(key,str):
        relative_custom_models[local_modelsfolder/key] = val;
    else:
        relative_custom_models[(local_modelsfolder/k for k in key)] = val;
custom_models = relative_custom_models;
# print(custom_models)

used_models = {};
ts = ["Nucleus", "Cell", "Separate - Cell + Nucleus models (two output folders)", "Combined - Cell + Nucleus models", "Separate - Single Model (two output folders)", "Combined - Single Model","Custom Models"]
ms = [{nucleus_model:"Nucleus"},{cell_model:"Cell"},
      {nucleus_model:"Nucleus",cell_model:"Cell"},
      {(nucleus_model,cell_model):"Combined"},
      {combined_model:("Nucleus","Cell")},
      {combined_model:"Combined"},
       custom_models]; 
target_model_map = {x:y for x,y in zip(ts,ms)};


print("using models:",list(target_model_map[segmentation_target].keys()));

out file: optotaxisbucket/movie_segmentation/2023.10.10 OptoTiam Exp 70/segmentation_output_masks
using models: [Path('gcp_transfer/fastai/iter3_4_nuc_continue.pkl'), Path('gcp_transfer/fastai/iter1_10_continue.pkl')]


## Helper Functions

In [26]:
def get_image_files_recursively(path):
    path = Path(path);
    out = get_image_files(path);
    subfiles = os.listdir(path);
    for f in subfiles:
        if (os.path.isdir(f)):
            out += get_image_files_recursively(path/f);
    return out;
            
def on_rm_error( func, path, exc_info):
    # path contains the path of the file that couldn't be removed
    # let's just assume that it's read-only and unlink it.
    os.chmod( path, stat.S_IWRITE )
    # os.unlink( path )

def cleardir(dir): #clears all files in dir without deleting dir
  for f in os.scandir(dir):
    # f = os.path.join(dir,f)
    if os.path.isdir(f): shutil.rmtree(f,onerror=on_rm_error); #just in case
    else: os.remove(f);    

# Segment Experiment
Will process all images in the segmentation input folder and continuously output segmented masks into the segmentation output masks folder in GCP.

### Pull GCP folders to local environment

In [27]:
print(gcp_modelsfolder)
print(gsutil_dest_folder)
if process_logging: logging.debug("copying remote folders...");

optotaxisbucket/models/fastai
gcp_transfer


In [28]:
#models folder
!gsutil -m cp -r -n "gs://{gcp_modelsfolder}" "{gsutil_dest_folder}"
# print(s)

Copying gs://optotaxisbucket/models/fastai/2x2_split_noaug_test.pkl...
Copying gs://optotaxisbucket/models/fastai/4x4_split_test.pkl...                
Copying gs://optotaxisbucket/models/fastai/5x5_context_split_test.pkl...        
Copying gs://optotaxisbucket/models/fastai/blah.c...                            
Copying gs://optotaxisbucket/models/fastai/e_cell_r1.pkl...                     
Copying gs://optotaxisbucket/models/fastai/e_nuc_r1.pkl...                      
Copying gs://optotaxisbucket/models/fastai/iter1_10_continue.pkl...             
Copying gs://optotaxisbucket/models/fastai/iter1_1_seg_redo.pkl...              
Copying gs://optotaxisbucket/models/fastai/iter1_2_seg_redo.pkl...              
Copying gs://optotaxisbucket/models/fastai/iter1_3_seg_redo.pkl...              
Copying gs://optotaxisbucket/models/fastai/iter1_4_seg_redo.pkl...              
Copying gs://optotaxisbucket/models/fastai/iter1_6_context_4_continue.pkl...    
Copying gs://optotaxisbucket/models/fa

In [29]:
# %%capture
#segmentation images
with capture_output() as capture:
    s=!gsutil -m cp -r -n "gs://{gcp_segmentation_images}" "{gsutil_dest_folder}"
if s[0].startswith('CommandException'):
    raise Exception("Error: unable to copy images:\n"+"\n".join(s));
else:
    print("Images successfully copied")
files_changed = False;
stitching_complete = False

Images successfully copied


### Loading a trained model

In [30]:
print("available cpu count:",len(os.sched_getaffinity(0)));

available cpu count: 16


In [31]:
if process_logging: logging.debug("preparing models...");
def label_func(x): return None; ##dummy function to make unpickling work, never used
def mask_from_image(x): return None; ##dummy function to make unpickling work, never used
model_outputs_dict = target_model_map[segmentation_target];
process_instructions = [];
output_dirs = [];
    
for model,outName in model_outputs_dict.items():
    key = [];
    if not isinstance(model,list):
        loadedlearner=load_learner(model,cpu=True);
        if torch.cuda.is_available():
            print("using gpu");
            loadedlearner.model = loadedlearner.model.cuda();
        key = [loadedlearner];
    else:
        for m in model:
            loadedlearner=load_learner(m,cpu=True);
            if torch.cuda.is_available():
                print("using gpu");
                loadedlearner.model = loadedlearner.model.cuda();
            key.append(loadedlearner);
    if not isinstance(outName,list):
        outName = [local_segmentation_output_masks/outName]
        output_dirs += outName;
    else:
        outName = [local_segmentation_output_masks/n for n in outName];
        output_dirs += outName;
    process_instructions.append((key,outName));
    
print("model loaded and instructions created");
        

model loaded and instructions created


### Do pre-segmentation file management
Create output folder, find which files are remaining to segment, chunk files into batches for segmentation and upload

In [32]:
if process_logging: logging.debug("preparing filepaths...");
if not os.path.exists(local_segmentation_output_masks):
    os.makedirs(local_segmentation_output_masks,exist_ok=True);
for d in output_dirs:
    if not os.path.exists(d):
        os.makedirs(d,exist_ok=True);
        

## prepare files

#as far as I can tell through testing, this only matters for ensuring RAM usage stays low
batchSize = 256;
if not external_splitting_stitching:
    batchSize = max(int(batchSize / (x_slices*y_slices) / loadedlearner.dls.bs)*loadedlearner.dls.bs,1);


files = get_image_files_recursively(local_segmentation_images);

partial_upload = False;

#make destination folders to match input directory tree
for name in files:
    for d in output_dirs:
        os.makedirs(d/(Path(os.path.relpath(name,local_segmentation_images)).parent),exist_ok=True);

completed_masks = [];
for d in progress_bar(output_dirs):
    cmasks = [os.path.relpath(x,d) for x in get_image_files_recursively(d)];
    if not persistent_files or True:
        c = !gsutil ls -r "gs://{gcp_segmentation_output_masks/os.path.basename(d)}"
        if any([x.startswith("CommandException") for x in c]): c = [];
        # print(c)
        t = [os.path.relpath(x,f"gs://{gcp_segmentation_output_masks/os.path.basename(d)}") for x in c if x != '' and not x.endswith((':','.zip','.nd','.flag'))];
        # print(t);
        partial_upload = partial_upload or any([s for s in t if t not in cmasks]);
        cmasks += t;
    completed_masks.append(cmasks);
# print(completed_masks);
print("input files:",len(files))
total_files = len(files);
print("output files:",[len(set(cmasks)) for cmasks in completed_masks])
# print("output files:",[cmasks for cmasks in completed_masks])

files = [fi for fi in files if any([os.path.relpath(fi,local_segmentation_images) not in cmasks for cmasks in completed_masks])];
print("remaining files:",len(files))
files_remaining = len(files);

# print(batchSize);
fileChunks = list(chunked(files, chunk_sz=batchSize));

filestate_read = True;


input files: 3468
output files: [0, 0]
remaining files: 3468


In [33]:
if external_splitting_stitching:
    if process_logging: logging.debug("starting external_splitting segmentation...");
    stitching_complete = False;
    if not filestate_read:
        raise Exception("Error: file output state not updated since last processing, please run the \"pre-segmentation file management\" cell again")
    filestate_read = False
    cpus = len(os.sched_getaffinity(0))
    if files_remaining != 0:
        files_changed = True;
        p = progress_bar(fileChunks,files_remaining//batchSize+1); #makes a progress bar
        for n,chunk in enumerate(p):
            for models,out in process_instructions:
                outputs = [];
                for m in models:
                    dl = m.dls.test_dl(chunk,num_workers=cpus);
                    if torch.cuda.is_available():
                        dl = m.dls.test_dl(chunk,num_workers=cpus,device='cuda');
                        dl.to('cuda');
                    _,_,dec_preds = m.get_preds(with_decoded=True,dl=dl);
                    outputs.append(dec_preds);
                if len(out) == len(outputs): #should also work for model splitting if can figure out how to make outputs the right shape
                    for p,put in zip(out,outputs):
                        for path,prediction in zip(chunk,put):
                            basename = os.path.relpath(path,local_segmentation_images); #preserve input directory structure
                            imsave(p/basename,prediction.numpy().astype('uint8'),check_contrast=False);
                else: #TODO: TEST AND FIX
                    print("ERROR: NOT IMPLEMENTED");
            if process_logging: 
                try:
                    time_remaining = format_time(p.pred_t - p.last_t + p.start_t);
                    logging.debug(f"chunk {n} complete; time remaining: {time_remaining}");
                except Exception as e:
                    logging.error(e);
                    logging.debug(f"chunk {n} complete");
                    
                
            if not persistent_files:
                with capture_output() as capture:
                    !gsutil -m rsync -r "{local_segmentation_output_masks}" "gs://{gcp_segmentation_output_masks}"
    if not stitching_complete:
        print("image segmentation process complete, finishing up...");
        with open(local_segmentation_output_masks/'segmentation_complete.flag','w') as f:
            pass;
        #zip files if required
        if zip_output:
            if not persistent_files or (partial_upload and files_changed):
                #download external files
                !gsutil -m rsync -r "gs://{gcp_segmentation_output_masks}" "{local_segmentation_output_masks}"
            for folder in output_dirs:
                if files_changed or not os.path.exists(folder.with_suffix('.zip')):
                    with capture_output() as capture:
                        !(cd "{folder.parent}" && zip -r "{folder.with_suffix('.zip').name}" "{folder.name}")
                        !gsutil -m cp "{folder.with_suffix('.zip')}" "gs://{gcp_segmentation_output_masks/folder.with_suffix('.zip').name}"
        #this is actually insane. The GCP bucket has to have at least one file in the parent directory
        #or it just like forgets to put the innermost directory. I have no idea why, but this fixes it
        with open("foothold.txt",'w') as f:
            pass;
        # with capture_output() as capture:
        #     !gsutil cp foothold.txt "gs://{gcp_segmentation_output_masks.parent}/foothold.txt"
        with capture_output() as capture:
            !gsutil -m rsync -r "{local_segmentation_output_masks}" "gs://{gcp_segmentation_output_masks}"
        print("folder processing complete");
        stitching_complete = True;
        if process_logging: logging.debug("segmentation fully complete");
        print("segmentation complete!");
    else:
        print("segmentation already complete!");
        stitching_complete = True;

In [ ]:
if not external_splitting_stitching:
    if process_logging: logging.debug("starting nonexternal_splitting segmentation");
    if not filestate_read:
        raise Exception("Error: file output state not updated since last processing, please run the \"pre-segmentation file management\" cell again")
    filestate_read = False
    from IPython.utils.io import capture_output
    from multiprocessing import Pool
    cpus = len(os.sched_getaffinity(0))
    
    def split_image(name):
        try:
            im = imread(name);
        except:
            raise IOError(f"Error while attempting to read image {name}");
        if auto_rescale:
            im = rescale_intensity(im);
        assert isinstance(im,np.ndarray);

        M = (im.shape[0]-context_bounds[0]-context_bounds[2]-crop[0]-crop[2])/y_slices;
        N = (im.shape[1]-context_bounds[1]-context_bounds[3]-crop[1]-crop[3])/x_slices;

        if int(M) != M or int(N) != N:
            raise Exception(f"ERROR: Mask with size {im.shape[:2]} cannot be sliced into {x_slices} columns and {y_slices} rows\nwith context bounds of {context_bounds}; {M} and {N} not integers");
        else:
            M = int(M)
            N = int(N)
            im = (im/256).astype('uint8');
            im = np.stack((im,im,im),axis=2);
            tiles = [im[y-context_bounds[0]:y+M+context_bounds[2],x-context_bounds[1]:x+N+context_bounds[3]] 
                    for y in range(context_bounds[0]+crop[0],im.shape[0]-crop[0]-crop[2]-context_bounds[0]-context_bounds[2],M) 
                    for x in range(context_bounds[1]+crop[1],im.shape[1]-crop[1]-crop[3]-context_bounds[1]-context_bounds[3],N)];
            # imBounds = [[y-context_bounds[0],y+M+context_bounds[2],x-context_bounds[1],x+N+context_bounds[3]] 
            #         for y in range(context_bounds[0]+crop[0],im.shape[0]-crop[0]-crop[2]-context_bounds[0]-context_bounds[2],M) 
            #         for x in range(context_bounds[1]+crop[1],im.shape[1]-crop[1]-crop[3]-context_bounds[1]-context_bounds[3],N)];
            return tiles;
    
    def stitch_image(tiles,outName):
        stitchMasks = []
        for i,m in enumerate(tiles):
            m = m.numpy().astype('uint8')
            y = i // x_slices;
            x = i % x_slices;
            imBounds = [crop[0]+context_bounds[0] if y != 0 else 0,m.shape[0]-crop[2]-context_bounds[2] if y != y_slices-1 else m.shape[0],crop[1]+context_bounds[1] if x != 0 else 0 ,m.shape[1]-crop[3]-context_bounds[3] if x != x_slices - 1 else m.shape[1]];
            stitchMasks.append(m[imBounds[0]:imBounds[1],imBounds[2]:imBounds[3]]);
        stitched = np.concatenate([np.concatenate(stitchMasks[i*x_slices:(i+1)*x_slices],axis=1) for i in range(y_slices)]);
        imsave(outName,stitched,check_contrast=False);
    if files_remaining != 0:
        files_changed = True;
        stitching_complete = False;
        prog = progress_bar(fileChunks,files_remaining//batchSize+1); #makes a progress bar
        for n,chunk in enumerate(prog):
            basenames = [os.path.relpath(c,local_segmentation_images) for c in chunk]; #preserve input directory structure
            # print(basenames);
            with Pool(cpus) as p:
                chunk = [item for sublist in p.map(split_image,chunk) for item in sublist];
            for models,out in process_instructions:
                outputs = [];
                for m in models:
                    dl = m.dls.test_dl(chunk,num_workers=cpus);
                    if torch.cuda.is_available():
                        dl = m.dls.test_dl(chunk,num_workers=cpus,device='cuda');
                        dl.to('cuda');
                    _,_,dec_preds = m.get_preds(with_decoded=True,dl=dl);
                    outputs.append(dec_preds);
                if len(out) == len(outputs): #should also work for model splitting if can figure out how to make outputs the right shape
                    for path,put in zip(out,outputs):
                        grouped_preds = [put[i:i + x_slices*y_slices] for i in range(0, len(put), x_slices*y_slices)]
                        saveNames = [path/b for b in basenames];
                        with Pool(cpus) as p:
                            p.starmap(stitch_image,zip(grouped_preds,saveNames));
                else:
                    print("ERROR: NOT IMPLEMENTED");
            if not persistent_files:
                with capture_output() as capture:
                    !gsutil -m rsync -r "{local_segmentation_output_masks}" "gs://{gcp_segmentation_output_masks}"
            if process_logging: 
                try:
                    time_remaining = format_time(prog.pred_t - prog.last_t + prog.start_t);
                    logging.debug(f"chunk {n} complete; time remaining: {time_remaining}");
                except Exception as e:
                    logging.error(e);
                    logging.debug(f"chunk {n} complete");
    if not stitching_complete:
        print("image segmentation process complete, finishing up...");
        with open(local_segmentation_output_masks/'segmentation_complete.flag','w') as f:
            pass;
        #zip files if required
        if zip_output:
            if not persistent_files or (partial_upload and files_changed):
                print("downloading remote files to zip...");
                #download external files
                with capture_output() as capture:
                    !gsutil -m rsync -r "gs://{gcp_segmentation_output_masks}" "{local_segmentation_output_masks}"
            print("zipping files...")
            for folder in progress_bar(output_dirs):
                if files_changed or not os.path.exists(folder.with_suffix('.zip')):
                    with capture_output() as capture:
                        !(cd "{folder.parent}" && zip -r "{folder.with_suffix('.zip').name}" "{folder.name}")
                        !gsutil -m cp "{folder.with_suffix('.zip')}" "gs://{gcp_segmentation_output_masks/folder.with_suffix('.zip').name}"
        print("copying files to remote...");
        with capture_output() as capture:
            !gsutil -m rsync -r "{local_segmentation_output_masks}" "gs://{gcp_segmentation_output_masks}";
        stitching_complete = True;
        if process_logging: logging.debug("segmentation complete, awaiting integrity verification");
        print("segmentation complete!");
    else:
        print("segmentation already complete!");
        stitching_complete = True;

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
#verify destination file integrity
#NOTE that this only compares the *number* of input files to the *number* of output files;
#filename mishaps could break this, as could errors when uploading the input files themselves.
original_files = !gsutil ls -r "gs://{gcp_segmentation_images}"
original_files = [os.path.relpath(x,f"gs://{gcp_segmentation_images}") for x in original_files if x and not x.endswith(('.flag','.nd','.zip',':'))];
# print(original_files);

#check output folder integrity
for d in progress_bar(output_dirs):
    c = !gsutil ls -r "gs://{gcp_segmentation_output_masks/os.path.basename(d)}"
    # print(c);
    if c[0].startswith('CommandException'):
        c = [];
    c = [x for x in c if x and not x.endswith((':','.flag','.zip','.nd')) and os.path.relpath(x,f"gs://{gcp_segmentation_output_masks/os.path.basename(d)}") in original_files];
    if len(c) < total_files:
        stitching_complete = False;
        if process_logging: logging.error(f"Error in segmentation or uploading: output dir {gcp_segmentation_output_masks/os.path.basename(d)} incomplete (expected {total_files} objects but got {len(c)}, please run file prep and segmentation cells again to ensure output integrity");
        print(f"Error in segmentation or uploading: output dir {gcp_segmentation_output_masks/os.path.basename(d)} incomplete (expected {total_files} objects but got {len(c)}, please run file prep and segmentation cells again to ensure output integrity");
#check zip file integrity, if applicable
if zip_output:
    ziplist = !gsutil ls -r "gs://{gcp_segmentation_output_masks}"
    ziplist = [os.path.basename(x) for x in ziplist if x.endswith('.zip')];
    for d in output_dirs:
        if d.with_suffix('.zip').name not in ziplist:
            stitching_complete = False;
            if process_logging: logging.error(f"Error in segmentation or uploading: output zip file {d.with_suffix('.zip').name} missing from output directory. Please run file prep and segmentation cells again to ensure output integrity");
            print(f"Error in segmentation or uploading: output zip file {d.with_suffix('.zip').name} missing from output directory. Please run file prep and segmentation cells again to ensure output integrity")
if stitching_complete:
    if logging: logging.debug("Segmentation successful! Output file integrity verified.");
    print("Segmentation successful! Output file integrity verified.")
else:
    print("Error: segmentation was not complete. Please run file prep and segmentation cells again to ensure output integrity");

In [ ]:
if stitching_complete:
    cleardir(GCP_transfer_folder)